In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# ==========================================
# 1. CẤU HÌNH & KHỞI TẠO
# ==========================================
NUM_CUSTOMERS = 400000
YEARS = [2018, 2019, 2020, 2021, 2022, 2023] # Dữ liệu 6 năm
np.random.seed(42)

print("1. Đang tạo hồ sơ tĩnh (Static Profile)...")
gender_array = np.random.choice(['M', 'F', 'O'], NUM_CUSTOMERS, p=[0.49, 0.49, 0.02])
age_array = np.random.normal(35, 10, NUM_CUSTOMERS).astype(int).clip(18, 70)

# Tạo Risk Score nội tại (Ẩn) - Quyết định "Số phận" khách hàng
# Dùng Gamma distribution để mô phỏng thực tế: Nhiều người tốt, ít người rất xấu
inherent_risk = np.random.gamma(shape=2, scale=1.0, size=NUM_CUSTOMERS) 
inherent_risk = (inherent_risk - inherent_risk.mean()) / inherent_risk.std() # Chuẩn hóa Z-score

# Trình độ & Hôn nhân
cond_edu = [age_array < 22, age_array <= 25]
choice_edu = [np.random.choice([1, 2], NUM_CUSTOMERS, p=[0.9, 0.1]), np.random.choice([1, 2, 3], NUM_CUSTOMERS, p=[0.3, 0.6, 0.1])]
default_edu = np.random.choice([1, 2, 3, 4], NUM_CUSTOMERS, p=[0.3, 0.5, 0.15, 0.05])
edu_array = np.select(cond_edu, choice_edu, default=default_edu)

prob_married = np.clip(((age_array - 22) / 30) + np.where(gender_array == 'F', 0.05, 0), 0.05, 0.98)
marital_status = np.where(np.random.rand(NUM_CUSTOMERS) < prob_married, 'Married', 'Single')

# Tạo AUM Gốc (Tài sản ban đầu)
# Logic: Tuổi cao, học vấn cao, rủi ro thấp -> Tài sản cao
log_aum = 14 + 0.03 * age_array + 0.4 * (edu_array >= 3) + 0.5 * (marital_status == 'Married') - 0.2 * inherent_risk + np.random.normal(0, 1.0, NUM_CUSTOMERS)
base_aum_init = np.exp(log_aum).astype(np.int64)

house_ownership = (np.random.rand(NUM_CUSTOMERS) < np.clip((age_array - 18) / 50, 0.05, 0.95)).astype(int)

ref_month = np.random.randint(1, 13, NUM_CUSTOMERS)
ref_day = np.random.randint(1, 29, NUM_CUSTOMERS)

customers = pd.DataFrame({
    'SOCIF': np.arange(1000000, 1000000 + NUM_CUSTOMERS),
    'C_GIOITINH': gender_array,
    'BASE_AGE': age_array,
    'TRINHDO': edu_array,
    'INITIAL_AUM': base_aum_init,
    'TTHONNHAN': marital_status,
    'SOHUUNHA': house_ownership,
    'NHANVIENBIDV': np.random.choice([0, 1], NUM_CUSTOMERS, p=[0.99, 0.01]),
    'INHERENT_RISK': inherent_risk,
    'REF_MONTH': ref_month, # <--- Cột mới: Tháng tham chiếu
    'REF_DAY': ref_day
})

# ==========================================
# 2. MỞ RỘNG DỮ LIỆU THEO NĂM
# ==========================================
print("2. Expand dữ liệu theo năm...")
growth_matrix = np.random.normal(1.08, 0.15, (NUM_CUSTOMERS, len(YEARS)))
growth_matrix[:, 2] -= 0.15 # 2020 Covid giảm
cum_growth_matrix = np.cumprod(growth_matrix, axis=1)

df_list = []
for i, year in enumerate(YEARS):
    temp_df = customers.copy()
    temp_df['year'] = year
    temp_df['BASE_AUM'] = (temp_df['INITIAL_AUM'] * cum_growth_matrix[:, i]).astype(np.int64).clip(0)
    
    # Biến động Risk theo vĩ mô (Năm xấu -> Risk nội tại tăng lên)
    macro_risk_adj = 0.5 if year in [2020, 2021] else 0
    temp_df['CURRENT_RISK'] = temp_df['INHERENT_RISK'] + np.random.normal(0, 0.5, NUM_CUSTOMERS) + macro_risk_adj
    
    df_list.append(temp_df)

df = pd.concat(df_list, ignore_index=True)
df = df.sort_values(by=['SOCIF', 'year'])
df['TUOI'] = df['BASE_AGE'] + (df['year'] - 2018)
df.drop(columns=['INITIAL_AUM', 'BASE_AGE'], inplace=True)

# ==========================================
# [MỚI - SỬA LẠI] 2.5 SINH SNAPSHOT_DATE CỐ ĐỊNH THEO MÙA
# ==========================================
print("2.5. Sinh ngày quan sát (Snapshot Date) theo chu kỳ cố định...")

# Logic: Kết hợp Năm hiện tại + Tháng cố định + Ngày cố định
# Sử dụng pd.to_datetime với dictionary để tạo ngày cực nhanh và chuẩn xác
df['SNAPSHOT_DATE'] = pd.to_datetime({
    'year': df['year'],
    'month': df['REF_MONTH'],
    'day': df['REF_DAY']
})

# Kiểm tra logic
print("   -> Kiểm tra tính nhất quán ngày tháng cho 1 khách hàng ngẫu nhiên:")
sample_cust = df['SOCIF'].iloc[0]
print(df[df['SOCIF'] == sample_cust][['SOCIF', 'year', 'SNAPSHOT_DATE']])

print(f"   -> Min Date: {df['SNAPSHOT_DATE'].min()}")
print(f"   -> Max Date: {df['SNAPSHOT_DATE'].max()}")

# ==========================================
# 3. SINH BIẾN TÀI CHÍNH & CẤU TRÚC NỢ
# ==========================================
print("3. Sinh biến tài chính & Cấu trúc nợ...")
N = len(df)

# --- 3.1 INCOME & CBAL ---
base_income = np.log1p(df['BASE_AUM']) * 1_200_000 * df['TRINHDO'].map({1:0.8, 2:1.0, 3:1.5, 4:2.5})
df['INCOME'] = (base_income * np.random.uniform(0.8, 1.5, N)).astype(int).clip(5_000_000)

# Loan Balance: Risk cao vay nhiều (Leverage cao)
leverage_factor = np.random.lognormal(0, 0.3, N) + (df['CURRENT_RISK'] * 0.2)
df['CBAL'] = (df['INCOME'] * 12 * leverage_factor * np.random.uniform(0.5, 2.0, N)).astype(int)

# Gán Zero Balance (25% khách hàng không vay hoặc đã tất toán)
mask_zero_debt = (df['INCOME'] < df['INCOME'].median()) & (np.random.rand(N) < 0.25)
df.loc[mask_zero_debt, 'CBAL'] = 0

# Limit & Org
df['CBALORG'] = (df['CBAL'] * np.random.uniform(1.0, 1.2, N)).astype(int)
df['AFLIMT_MAX'] = (np.maximum(df['CBALORG'], df['INCOME']*24) * np.random.uniform(1.0, 1.5, N)).astype(int)
df['AFLIMT_MIN'] = (df['AFLIMT_MAX'] * 0.9).astype(int)
df['AFLIMT_AVG'] = (df['AFLIMT_MAX'] + df['AFLIMT_MIN']) // 2

# Max/Min/Avg
df['CBAL_AVG'] = (df['CBAL'] * np.random.uniform(0.8, 1.2, N)).astype(int)
df['CBAL_MAX'] = np.maximum.reduce([(df['CBAL_AVG']*1.2).astype(int), df['CBAL'], df['CBAL_AVG']])
df['CBAL_MIN'] = np.minimum.reduce([(df['CBAL_AVG']*0.8).astype(int), df['CBAL'], df['CBAL_AVG']])

# --- 3.2 COLLATERAL & LTV ---
df['COLLATERAL_VALUE'] = (df['BASE_AUM'] * 0.8 + df['SOHUUNHA'] * 2e9).astype(int)
df['LTV'] = np.where(df['COLLATERAL_VALUE'] > 0, (df['CBAL'] / df['COLLATERAL_VALUE']) * 100, 100)
df['LTV'] = df['LTV'].clip(0, 200)

# --- 3.3 DEPOSIT (Thanh khoản) ---
# Risk cao -> Ít tiền gửi tích lũy
df['N_AVG_DEPOSIT_12M'] = (df['BASE_AUM'] * np.random.uniform(0.05, 0.5, N) / np.exp(df['CURRENT_RISK']*0.5)).astype(int)
df['N_AVG_DEPOSIT_6M'] = df['N_AVG_DEPOSIT_12M']
df['N_AVG_DD_12M'] = (df['N_AVG_DEPOSIT_12M'] * 0.2).astype(int)
df['N_AVG_CD_12M'] = df['N_AVG_DEPOSIT_12M'] - df['N_AVG_DD_12M']
df['FLAG_SALARY_ACC'] = np.where(df['INCOME'] > 15_000_000, 1, 0)
df['FLAG_DEPOSIT'] = np.where(df['N_AVG_DEPOSIT_12M'] > 1_000_000, 1, 0)

# --- 3.4 CẤU TRÚC KỲ HẠN NỢ (SHORT/LONG TERM) ---
# [QUAN TRỌNG] Phải tính cái này trước khi tính Duration
w_short = np.random.uniform(0, 1, N)
df['CBAL_SHORTTERM_LOAN'] = (df['CBAL'] * w_short).astype(int)
df['CBAL_LONGTERM_LOAN'] = df['CBAL'] - df['CBAL_SHORTTERM_LOAN']
df['HAS_SHORTTERM_LOAN'] = np.where(df['CBAL_SHORTTERM_LOAN'] > 0, 1, 0)
df['HAS_LONGTERM_LOAN'] = np.where(df['CBAL_LONGTERM_LOAN'] > 0, 1, 0)

# ==========================================
# 4. SINH BIẾN DURATION & CÁC BIẾN KHÁC
# ==========================================
print("4. Sinh biến kỳ hạn (Duration) & Lãi suất...")

# Logic: Random cơ bản (6-24 tháng cho vay ngắn hạn)
df['DURATION_MAX'] = np.random.choice([6, 9, 12, 18, 24], N, p=[0.1, 0.1, 0.4, 0.2, 0.2])

# Logic Override: Nếu có dư nợ dài hạn -> Kỳ hạn phải dài (3-20 năm)
mask_long = (df['CBAL_LONGTERM_LOAN'] > 10_000_000)
num_long = mask_long.sum()
if num_long > 0:
    df.loc[mask_long, 'DURATION_MAX'] = np.random.choice(
        [36, 60, 120, 180, 240], num_long, p=[0.3, 0.3, 0.2, 0.1, 0.1]
    )

# Tính thời gian còn lại & thời gian đã qua
df['REMAINING_DURATION_MAX'] = (df['DURATION_MAX'] * np.random.uniform(0.1, 0.95, N)).astype(int)
df['TIME_TO_OP_MAX'] = df['DURATION_MAX'] - df['REMAINING_DURATION_MAX']

# Lãi suất (Risk cao -> Lãi cao)
df['RATE_AVG'] = np.random.normal(9.0, 1.5, N) + (df['CURRENT_RISK'] * 2.0)
df['RATE_AVG'] = df['RATE_AVG'].clip(5, 25)

# Purpose Code (Mục đích vay)
df['PURCOD_MAX'] = np.random.choice([1, 2, 3, 6, 9], N, p=[0.2, 0.1, 0.1, 0.5, 0.1])
df['PURCOD_MIN'] = df['PURCOD_MAX']

# ==========================================
# 5. SINH BIẾN HÀNH VI (HISTORICAL BEHAVIOR)
# ==========================================
print("5. Sinh biến hành vi (DPD, Overdue)...")

# Base DPD sinh ra từ Risk Score & Áp lực tài chính (LTV)
lambda_dpd = np.exp(df['CURRENT_RISK'] - 2) # Risk cao -> Lambda cao
raw_dpd = np.random.exponential(scale=lambda_dpd) * 10 

stress_factor = (df['LTV'] / 80) + (1e8 / (df['N_AVG_DEPOSIT_12M'] + 1))
df['MAX_DPD_12M'] = (raw_dpd * stress_factor).astype(int).clip(0, 900)

# Observed DPD (Thêm nhiễu)
df['MAX_DPD_12M_OBS'] = df['MAX_DPD_12M'] 
df['AVG_OD_DPD_12M'] = (df['MAX_DPD_12M_OBS'] * np.random.uniform(0.1, 0.5, N)).astype(int)
df['SUM_ALL_OD_12M'] = (df['MAX_DPD_12M_OBS'] * np.random.uniform(1, 3, N)).astype(int)

# Trạng thái HIỆN TẠI (Dùng để lọc)
df['BAD_CURRENT'] = np.where(df['MAX_DPD_12M_OBS'] >= 90, 1, 0)
df['XULYNO'] = np.where((df['MAX_DPD_12M_OBS'] >= 180) & (np.random.rand(N) < 0.6), 1, 0)

# Nhóm nợ CIC (Logic chuẩn)
conditions = [
    df['MAX_DPD_12M_OBS'] >= 360,
    df['MAX_DPD_12M_OBS'] >= 180,
    df['MAX_DPD_12M_OBS'] >= 90,
    df['MAX_DPD_12M_OBS'] >= 10
]
choices = [5, 4, 3, 2]
df['MAX_NHOMNOCIC'] = np.select(conditions, choices, default=1)

df['N_AVG_OVERDUE_CBAL_12M'] = np.where(df['MAX_DPD_12M_OBS'] > 0, df['CBAL']*0.5, 0).astype(int)
df['CBAL_TO_INC_12MON'] = df['CBAL'] / (df['INCOME'] + 1)

# Biến Macro
macro_data = {2018: 7.08, 2019: 7.02, 2020: 2.91, 2021: 2.58, 2022: 8.02, 2023: 5.05}
df['REAL_GDP_GROWTH_12M'] = df['year'].map(macro_data)

# ==========================================
# 6. SINH TARGET (TUNED FOR MODELING)
# ==========================================
print("6. Sinh Target tương lai (Probability Model)...")

# Công thức xác suất (Nhân quả: Behavior + Financial -> Target)
# Trọng số đã được tinh chỉnh để Correlation > 0.3
score_behavior = (
    0.08 * df['MAX_DPD_12M_OBS'] +       # DPD cao -> Rủi ro
    1.5  * (df['MAX_NHOMNOCIC'] >= 2)    # Đã nhóm 2 -> Rủi ro
)

score_financial = (
    0.03 * df['LTV'] +                   # LTV cao -> Rủi ro
    0.8 * (df['CBAL_TO_INC_12MON'] > 8) - # Nợ quá nhiều
    0.6 * np.log1p(df['N_AVG_DEPOSIT_12M']) # Có tiền gửi -> Giảm rủi ro
)

# Random Shock (Giảm nhẹ noise để tín hiệu rõ hơn)
random_shock = np.random.normal(0, 1.5, N)

# Intercept -4.2 để kéo Bad Rate lên khoảng 8-10%
logit_p = -4.2 + score_behavior + score_financial + random_shock
prob_default = 1 / (1 + np.exp(-logit_p))

df['BAD_NEXT_12M'] = (np.random.rand(N) < prob_default).astype(int)

# ==========================================
# 7. LỌC DỮ LIỆU (CLEANING)
# ==========================================
print("7. Lọc bỏ hồ sơ rác & Đã xấu (Population Filter)...")

cond_remove = (
    (df['BAD_CURRENT'] == 1) |      # Loại người đang Bad
    (df['MAX_NHOMNOCIC'] >= 3) |    # Loại nhóm nợ xấu
    (df['XULYNO'] == 1) |           # Loại đã xử lý nợ
    (df['TUOI'] < 18) |             # Loại trẻ em
    (df['CBAL'] <= 0) |             # Loại không có dư nợ (Inactive)
    (df['INCOME'] <= 0)
)

df_clean = df[~cond_remove].copy()

# Gán nhãn Sample (Train/OOS/OOT)
def assign_sample(year):
    if year <= 2021: return 'TRAIN'
    if year == 2022: return 'OOS'
    return 'OOT' # 2023

df_clean['SAMPLE_TYPE'] = df_clean['year'].apply(assign_sample)

# ==========================================
# 8. KIỂM TRA KẾT QUẢ
# ==========================================
print("=" * 40)
print(f"Kích thước gốc: {df.shape}")
print(f"Kích thước sau lọc (Sạch): {df_clean.shape}")
print("-" * 30)
print("Tỷ lệ Bad Rate (Target) trên tập sạch:")
print(df_clean.groupby('SAMPLE_TYPE')['BAD_NEXT_12M'].mean())
print("-" * 30)
print("Check Correlation (Kỳ vọng: DPD > 0.3, LTV > 0.1):")
cols_check = ['MAX_DPD_12M_OBS', 'LTV', 'N_AVG_DEPOSIT_12M', 'BASE_AUM', 'BAD_NEXT_12M']
print(df_clean[cols_check].corr()['BAD_NEXT_12M'])
print("=" * 40)

# Lưu lại DataFrame cuối cùng
df = df_clean.copy()

1. Đang tạo hồ sơ tĩnh (Static Profile)...
2. Expand dữ liệu theo năm...
2.5. Sinh ngày quan sát (Snapshot Date) theo chu kỳ cố định...
   -> Kiểm tra tính nhất quán ngày tháng cho 1 khách hàng ngẫu nhiên:
           SOCIF  year SNAPSHOT_DATE
0        1000000  2018    2018-07-08
400000   1000000  2019    2019-07-08
800000   1000000  2020    2020-07-08
1200000  1000000  2021    2021-07-08
1600000  1000000  2022    2022-07-08
2000000  1000000  2023    2023-07-08
   -> Min Date: 2018-01-01 00:00:00
   -> Max Date: 2023-12-28 00:00:00
3. Sinh biến tài chính & Cấu trúc nợ...
4. Sinh biến kỳ hạn (Duration) & Lãi suất...
5. Sinh biến hành vi (DPD, Overdue)...
6. Sinh Target tương lai (Probability Model)...
7. Lọc bỏ hồ sơ rác & Đã xấu (Population Filter)...
Kích thước gốc: (2400000, 52)
Kích thước sau lọc (Sạch): (1138159, 53)
------------------------------
Tỷ lệ Bad Rate (Target) trên tập sạch:
SAMPLE_TYPE
OOS      0.112441
OOT      0.111042
TRAIN    0.126373
Name: BAD_NEXT_12M, dtype: float

In [2]:
df.to_csv('gen_data.csv', index=False)

In [3]:
df.columns

Index(['SOCIF', 'C_GIOITINH', 'TRINHDO', 'TTHONNHAN', 'SOHUUNHA',
       'NHANVIENBIDV', 'INHERENT_RISK', 'REF_MONTH', 'REF_DAY', 'year',
       'BASE_AUM', 'CURRENT_RISK', 'TUOI', 'SNAPSHOT_DATE', 'INCOME', 'CBAL',
       'CBALORG', 'AFLIMT_MAX', 'AFLIMT_MIN', 'AFLIMT_AVG', 'CBAL_AVG',
       'CBAL_MAX', 'CBAL_MIN', 'COLLATERAL_VALUE', 'LTV', 'N_AVG_DEPOSIT_12M',
       'N_AVG_DEPOSIT_6M', 'N_AVG_DD_12M', 'N_AVG_CD_12M', 'FLAG_SALARY_ACC',
       'FLAG_DEPOSIT', 'CBAL_SHORTTERM_LOAN', 'CBAL_LONGTERM_LOAN',
       'HAS_SHORTTERM_LOAN', 'HAS_LONGTERM_LOAN', 'DURATION_MAX',
       'REMAINING_DURATION_MAX', 'TIME_TO_OP_MAX', 'RATE_AVG', 'PURCOD_MAX',
       'PURCOD_MIN', 'MAX_DPD_12M', 'MAX_DPD_12M_OBS', 'AVG_OD_DPD_12M',
       'SUM_ALL_OD_12M', 'BAD_CURRENT', 'XULYNO', 'MAX_NHOMNOCIC',
       'N_AVG_OVERDUE_CBAL_12M', 'CBAL_TO_INC_12MON', 'REAL_GDP_GROWTH_12M',
       'BAD_NEXT_12M', 'SAMPLE_TYPE'],
      dtype='object')